In [1]:
%load_ext lab_black

In [1]:
import pandas as pd
from nltk import tokenize
import boto3
import botocore
from datetime import datetime
import matplotlib.pylab as plt
import yfinance as yf
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from numba import jit, prange

In [2]:
# Load appropriate packages for sentiment analysis
from flair.models import TextClassifier
from flair.models import SequenceTagger
from flair.data import Sentence
from segtok.segmenter import split_single
import pandas as pd
import re


classifier = TextClassifier.load("en-sentiment")

ModuleNotFoundError: No module named 'flair'

In [2]:
nvidia_cleaned_df = pd.read_csv("sentiment_test.gz", compression="gzip")

In [7]:
nvidia_cleaned_df.searchedDate.min()

'2020-10-02 01:45:21.553469+00:00'

In [8]:
nvidia_cleaned_df.searchedDate.max()

'2021-02-20 21:30:42.127197+00:00'

In [9]:
nvidia_cleaned_df.loc[0,'captionString']

"so this setup that you're seeing is what we had to do to get 3090s and sli functional this early in the game 3090 is an sli turns out not so simple to set up right now there's a number of various issues we ran into like the sli fingers on the cards not lining up it's not standardized so they're in slightly different places on different boards and not just vertically this time either but also horizontally on the board and also the limitations for sli these days are more strict than ever as nvidia has officially dropped sli support going forward and so now only things with explicit multi gpu support or mgpu support will actually run two cars and we had trouble running some of these games that don't technically require a bridge without a bridge we had the sli bridge in there in order to get it all working but it's there and we'll have some 30 90 sli benchmarks today for the highest performing possible gaming setup but one which is also not at all worth the money but we'll talk about that

In [13]:
# split into words
from nltk.tokenize import word_tokenize
tokens = word_tokenize(nvidia_cleaned_df.loc[0,'captionString'])
# remove all tokens that are not alphabetic
words = [word for word in tokens if word.isalpha()]
print(words[:100])

['so', 'this', 'setup', 'that', 'you', 'seeing', 'is', 'what', 'we', 'had', 'to', 'do', 'to', 'get', 'and', 'sli', 'functional', 'this', 'early', 'in', 'the', 'game', 'is', 'an', 'sli', 'turns', 'out', 'not', 'so', 'simple', 'to', 'set', 'up', 'right', 'now', 'there', 'a', 'number', 'of', 'various', 'issues', 'we', 'ran', 'into', 'like', 'the', 'sli', 'fingers', 'on', 'the', 'cards', 'not', 'lining', 'up', 'it', 'not', 'standardized', 'so', 'they', 'in', 'slightly', 'different', 'places', 'on', 'different', 'boards', 'and', 'not', 'just', 'vertically', 'this', 'time', 'either', 'but', 'also', 'horizontally', 'on', 'the', 'board', 'and', 'also', 'the', 'limitations', 'for', 'sli', 'these', 'days', 'are', 'more', 'strict', 'than', 'ever', 'as', 'nvidia', 'has', 'officially', 'dropped', 'sli', 'support', 'going']


In [14]:
# remove punctuation from each word
import string
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in tokens]
# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]
# filter out stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
print(words[:100])

['setup', 'seeing', 'get', 'sli', 'functional', 'early', 'game', 'sli', 'turns', 'simple', 'set', 'right', 'number', 'various', 'issues', 'ran', 'like', 'sli', 'fingers', 'cards', 'lining', 'standardized', 'slightly', 'different', 'places', 'different', 'boards', 'vertically', 'time', 'either', 'also', 'horizontally', 'board', 'also', 'limitations', 'sli', 'days', 'strict', 'ever', 'nvidia', 'officially', 'dropped', 'sli', 'support', 'going', 'forward', 'things', 'explicit', 'multi', 'gpu', 'support', 'mgpu', 'support', 'actually', 'run', 'two', 'cars', 'trouble', 'running', 'games', 'nt', 'technically', 'require', 'bridge', 'without', 'bridge', 'sli', 'bridge', 'order', 'get', 'working', 'sli', 'benchmarks', 'today', 'highest', 'performing', 'possible', 'gaming', 'setup', 'one', 'also', 'worth', 'money', 'talk', 'video', 'brought', 'team', 'tforce', 'cardea', 'ceramic', 'ssd', 'available', 'one', 'terabyte', 'two', 'terabyte', 'capacities', 'pcie', 'support', 'high']


In [19]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
stemmed = [porter.stem(word) for word in words]
print(stemmed[:200])

['setup', 'see', 'get', 'sli', 'function', 'earli', 'game', 'sli', 'turn', 'simpl', 'set', 'right', 'number', 'variou', 'issu', 'ran', 'like', 'sli', 'finger', 'card', 'line', 'standard', 'slightli', 'differ', 'place', 'differ', 'board', 'vertic', 'time', 'either', 'also', 'horizont', 'board', 'also', 'limit', 'sli', 'day', 'strict', 'ever', 'nvidia', 'offici', 'drop', 'sli', 'support', 'go', 'forward', 'thing', 'explicit', 'multi', 'gpu', 'support', 'mgpu', 'support', 'actual', 'run', 'two', 'car', 'troubl', 'run', 'game', 'nt', 'technic', 'requir', 'bridg', 'without', 'bridg', 'sli', 'bridg', 'order', 'get', 'work', 'sli', 'benchmark', 'today', 'highest', 'perform', 'possibl', 'game', 'setup', 'one', 'also', 'worth', 'money', 'talk', 'video', 'brought', 'team', 'tforc', 'cardea', 'ceram', 'ssd', 'avail', 'one', 'terabyt', 'two', 'terabyt', 'capac', 'pcie', 'support', 'high', 'transfer', 'speed', 'team', 'claim', 'ssd', 'sequenti', 'speed', 'gigabyt', 'per', 'second', 'read', 'gigabyt

Vader Sentiment

In [6]:
# load vader sentiment for comparing the uncleaned strings to the modified ones
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [29]:
nvidia_cleaned_df.columns

Index(['Unnamed: 0', 'Index', 'videoID', 'datePub', 'searchedDate',
       'VideoTitle', 'channelTitle', 'viewCount', 'likeCount', 'dislikeCount',
       'captionString'],
      dtype='object')

In [33]:
print(f'Title to score = {nvidia_cleaned_df["captionString"][5]}')
score = sid.polarity_scores(str(nvidia_cleaned_df["captionString"][5]))
print(score)

Title to score = (clapboard sounds) - Did you guys just seethe same thing I did? RTX 3070 is gonna outperform a 2080Ti at less than half the price, 499. Is Nvidia insane. I mean, even with all of the leaks indicating that Ampere wasgonna be super powerful, I was in no way ready for this, so okay. (sighs) Deep breath, there's alot to get through here, so let's get started. After an F in the chat forany recent 2080Ti buyers. (giggles) Ouch, like dang. Oh and also our sponsor. Ridge Wallet wants to redefine the wallet with its compact frameand RFID blocking plates. Check out how they can helpkeep your wallet bulge down and use our offer code Linus to save 10% and getfree worldwide shipping. (upbeat music) Now, obviously I'm notready to hand Nvidia their crown until we seethe new cards for ourselves. But based on what they've shown, we are in for one heck of a ride. They announced well a lot andwe're gonna get to all of it, but the big one is, Ampere is real and Ampere is an absolute monst

In [52]:
# utilize regular expressions to split large stings into sentences that are centered around the key words
import re

text = " allet wants to redefine the wallet with its compact frameand RFID blocking plates. Check out how they can helpkeep your wallet bulge down and use our offer code Linus to save 10% and getfree worldwide shipping. (upbeat music) Now, obviously I'm notready to hand Nvidia their crown until we seethe new cards for ourselves. But based on what they've shown, we are in for one heck of a ride. They announced well a lot andwe're gonna get to all of it, but the big one is, Ampere is real and Ampere is an absolute monster. As it turns out, the recentleaks were accurate in many if not most ways."


def search(target, text, context=6):
    # It's easier to use re.findall to split the string,
    # as we get rid of the punctuation
    words = re.findall(r"\w+", text)

    matches = (i for (i, w) in enumerate(words) if w.lower() == target)
    for index in matches:
        if index < context // 2:
            yield words[0 : context + 1]
        elif index > len(words) - context // 2 - 1:
            yield words[-(context + 1) :]
        else:
            yield words[index - context // 2 : index + context // 2 + 1]


print(list(search("nvidia", text, 10)))

[['I', 'm', 'notready', 'to', 'hand', 'Nvidia', 'their', 'crown', 'until', 'we', 'seethe']]


In [53]:
search("nvidia", text, 15)

<generator object search at 0x00000201143A05C8>

In [59]:
test_list_to_str = list(search("nvidia", text, 30))
test_list_to_str

[['save',
  '10',
  'and',
  'getfree',
  'worldwide',
  'shipping',
  'upbeat',
  'music',
  'Now',
  'obviously',
  'I',
  'm',
  'notready',
  'to',
  'hand',
  'Nvidia',
  'their',
  'crown',
  'until',
  'we',
  'seethe',
  'new',
  'cards',
  'for',
  'ourselves',
  'But',
  'based',
  'on',
  'what',
  'they',
  've']]

In [60]:
new_str = " ".join(test_list_to_str[0])

In [61]:
score = sid.polarity_scores(new_str)
print(score)

{'neg': 0.0, 'neu': 0.808, 'pos': 0.192, 'compound': 0.7506}


### Flair Sentiment

In [64]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load("en-sentiment")
sentence = Sentence(new_str)
classifier.predict(sentence)
# print sentence with predicted labels
print("Sentence above is: ", sentence.labels)

2021-03-02 20:27:50,602 loading file C:\Users\Chris\.flair\models\sentiment-en-mix-distillbert_3.1.pt
Sentence above is:  [POSITIVE (0.8555)]
